In [1]:
'''
Copyright (c) 2019, Crystal Xue
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
3. All advertising materials mentioning features or use of this software
   must display the following acknowledgement:
   This product includes software developed by the <organization>.
4. Neither the name of the <organization> nor the
   names of its contributors may be used to endorse or promote products
   derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY CRYSTAL XUE ''AS IS'' AND ANY
EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL CRYSTAL XUE BE LIABLE FOR ANY
DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
'''

"\nCopyright (c) 2019, Crystal Xue\nAll rights reserved.\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are met:\n1. Redistributions of source code must retain the above copyright\n   notice, this list of conditions and the following disclaimer.\n2. Redistributions in binary form must reproduce the above copyright\n   notice, this list of conditions and the following disclaimer in the\n   documentation and/or other materials provided with the distribution.\n3. All advertising materials mentioning features or use of this software\n   must display the following acknowledgement:\n   This product includes software developed by the <organization>.\n4. Neither the name of the <organization> nor the\n   names of its contributors may be used to endorse or promote products\n   derived from this software without specific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY CRYSTAL XUE ''AS IS'' AND AN

In [2]:
# This cell downloads necessary packages, which contain functions that the program will call later to scrape necessary information.
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import numpy as np

In [3]:
def get_headers():
    # Creating headers.
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    return headers

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, headers=get_headers(), stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
def get_address_price(html, isDebug=True):
    # Scrapes desired information from html code. Can be modified to include more/less information as long as content is in html file.
    addressPriceList = list()
    na = 'n/a'
    i = 0
    for li in html.select('li[class="component_property-card js-component_property-card "]'):
        # Selects each individual property
        i = i+1
        
        # Collects street address
        streetAddress = li.select('span[class="listing-street-address"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(streetAddress)
        
        # Collects city
        city = li.select('span[class="listing-city"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(city)
         
        # Collects state (region for the possibility of international use)
        region = li.select('span[class="listing-region"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(region)
        
        # Collects postal code
        postal = li.select('span[class="listing-postal"]')[0].text.strip(' ,\t\n\r')
        if isDebug:
            print(postal)
        
        # Collects price displayed. Could be lower bound, range, etc.
        if bool(li.select('span[class="data-price"]')) == True:
            dataPrice = li.select('span[class="data-price"]')[0].text.strip(' \t\n\r')
        else:
            dataPrice = na
        if isDebug:
            print("dataPrice is empty")
        
        if bool(li.select('span[class="price-addon-text"]')) == True:
            priceAddonText = li.select('span[class="price-addon-text"]')[0].text.strip(' \t\n\r')
        else:
            priceAddonText = na
        if isDebug:
            print("priceAddonText is empty")
            
        # Collects longitude and latitude
        lat = ''
        lon = ''
        
        if bool(li.select('span[class="listing-geo"]')) == True:
            for tag in li.select('span[class="listing-geo"]')[0].find_all("meta"):
                if tag.get("itemprop", None) == "latitude":
                    lat = tag.get("content", None)
                if tag.get("itemprop", None) == "longitude":
                    lon = tag.get("content", None)
                    
        # Collects property type
        if bool(li.select('div[class="property-type"]')) == True:
            propType = li.select('div[class="property-type"]')[0].text.strip(' \t\n\r')
            propType = propType.replace(' ','')
            propType = propType.replace('forRent','')
        else:
            propType = na
        if isDebug:
            print("propType is empty")
        
        # Collects bedrooms
        if bool(li.select('li[data-label="property-meta-beds"]')) == True:    
            bed = li.select('li[data-label="property-meta-beds"]')[0].text.strip(' ,\t\n\r')
            bed = bed.replace(' ','')
            bed = bed.replace('bed','')
        else:
            bed = na
        if isDebug:
            print("bed is empty")
            
        # Collects bathrooms
        if bool(li.select('li[data-label="property-meta-baths"]')) == True:
            bath = li.select('li[data-label="property-meta-baths"]')[0].text.strip(' ,\t\n\r')
            bath = bath.replace(' ','')
            bath = bath.replace('bath','')
        else:
            bath = na
        if isDebug:
            print("bath is empty")
            
       # Collects sqft
        if bool(li.select('li[data-label="property-meta-sqft"]')) == True:
            sqft = li.select('li[data-label="property-meta-sqft"]')[0].text.strip(' ,\t\n\r')
            sqft = sqft.replace(' ','')
            sqft = sqft.replace('sqft','')
        else:
            sqft = na
        if isDebug:
            print(sqft)
            
        source = "Realtor.com"
        
        mode = "buy"
     
        # Determine if property has Dayton address (based on city field)
        if(city.upper().find("DAYTON") >= 0):
            addressPriceList.append([streetAddress, city, region, postal, lat, lon, dataPrice, priceAddonText, propType, bed, bath, sqft, mode, source])
        else:
            if isDebug:
                print('out city: ', [streetAddress, city, region, postal, lat, lon, dataPrice, priceAddonText, propType, bed, bath, sqft, mode, source])
            continue

    # Print number of properties on page. Should be used as a check to make sure scraper is passing through all properties.
    print('# cards in page: ', i)
    
    # Print number of properties listed in Dayton, OH. These properties will be written to csv file.
    print('# cards in city: ', len(addressPriceList))
    
    return addressPriceList
    
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(i))

In [5]:
#Isolates and call information for each individual property
def get_property_card(city, state, page=1, isDebug=True):
    
    url = 'https://www.realtor.com/realestateandhomes-search/' + city + '_' + state + '/radius-1/pg-' + str(page)
    print(url)
    
    '''
    Use this url to make debugging easier. This way, the scraper does not cycle through every single page.
    
    url='https://www.realtor.com/realestateandhomes-search/Dayton_OH/radius-1/pg-1'
    '''
    
    print(url)
    
    '''
    Use this code for debugging. Code returns html file of page and stops. Use it isolate pieces of information and ensure return is html.
    
    response = simple_get(url)
    html = BeautifulSoup(response, 'html.parser')
    return html
    '''

    response = simple_get(url)

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        if isDebug:
            print(html)
            
        return get_address_price(html, isDebug)
    
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))

In [6]:
# Test cell. Ensure scraper is running properly by testing with first page.
property_cards = get_property_card('Dayton', 'OH', 1, isDebug=True)

https://www.realtor.com/realestateandhomes-search/Dayton_OH/radius-1/pg-1
https://www.realtor.com/realestateandhomes-search/Dayton_OH/radius-1/pg-1
<!DOCTYPE html>
<html lang="en"><head><script>window.RDCXCDN='/assets/prod/7cb5158';if (!window.rdcxreferrer) {window.rdcxreferrer = document.referrer;}window.dtmVersion='160';</script><script src="/experiments.js?ts=1563810305009" type="text/javascript"></script><script>
                (function() {
                  window.markAs = function() {
                    var perfAPI = window.performance;
                    if (perfAPI && perfAPI.mark) {
                      for (var i = 0; i < arguments.length; i++) {
                        perfAPI.mark(arguments[i])
                      }
                    }
                  };
                  window.publishEvent = function(e_name){
                    var event;
                    if(typeof Event === "function"){
                      event = new Event(e_name)
                    }


In [7]:
# Test cell. Ensure scraper is only returning properties listed in Dayton, OH.
print(property_cards)

[]


In [8]:
# Test cell. Count output of previous cell and check for match.
len(property_cards)

0

In [9]:
# Run scraper for all addresses in Dayton
totalAddressList = list()
i = 1
while True:
    addresses = get_property_card('Dayton', 'OH', i, isDebug=False)
    print(addresses)
    l = len(addresses)
    print(l)
    print(i)
    if l > 0 :
        totalAddressList = totalAddressList + addresses
        i = i + 1
    else:
        break
        
print(totalAddressList)
print(len(totalAddressList))

https://www.realtor.com/realestateandhomes-search/Dayton_OH/radius-1/pg-1
https://www.realtor.com/realestateandhomes-search/Dayton_OH/radius-1/pg-1
# cards in page:  0
# cards in city:  0
[]
0
1
[]
0


In [10]:
# Test cell. Double-check that totalAddressList is in form that can be written to csv file. Should see "list" in output.
type(totalAddressList)

list

In [11]:
# Create csv file. Headers can be modified to refer to actual values in list.
import csv

header = ['streetAddress', 'city', 'region', 'postal', 'lat', 'lon', 'dataPrice', 'priceAddonText', 'propType', 'bed', 'bath', 'sqft', 'mode', 'source']
type(header)

list

In [12]:
# Write totalAddressList to csv file and download csv file.
with open('realtorBuy.csv', 'w') as f:
    csv_writer = csv.writer(f)
 
    csv_writer.writerow(header) # write header
 
    for row in totalAddressList:
        csv_writer.writerow(row)
        
print("Done")

Done
